In [424]:
from pathlib import Path

from neo4j import GraphDatabase
from tqdm.notebook import tqdm
import pandas as pd

In [425]:
# log in to neo4j db
URI = "bolt://localhost:7687"
driver = GraphDatabase.driver(URI, auth=("neo4j", "password"))

In [426]:
def check_connection(driver):
    with driver.session() as session:
        result = session.run("CALL dbms.components()")
        for record in result:
            print(record)


check_connection(driver)

<Record name='Neo4j Kernel' versions=['5.26.0'] edition='community'>


In [427]:
with driver.session() as session:
    count_query = "MATCH (n) RETURN count(n) as total"
    total_nodes = session.run(count_query).single()["total"]

    progress = tqdm(total=total_nodes, desc="Deleting nodes", unit="nodes")

    # Delete in batches
    batch_size = 250
    while True:
        delete_query = """
        MATCH (n)
        WITH n LIMIT $batch_size
        DETACH DELETE n
        RETURN count(n) as deleted
        """

        result = session.run(delete_query, batch_size=batch_size)
        deleted_count = result.single()["deleted"]

        progress.update(deleted_count)

        if deleted_count == 0:
            break

    progress.close()

Deleting nodes:   0%|          | 0/2500 [00:00<?, ?nodes/s]

In [428]:
DATASET_PATH = Path("dataset")
airlines_csv = DATASET_PATH / "airlines.csv"
countries_csv = DATASET_PATH / "countries.csv"
planes_csv = DATASET_PATH / "planes.csv"
routes_csv = DATASET_PATH / "routes.csv"
terminals_csv = DATASET_PATH / "terminals.csv"

In [429]:
airlines_df = pd.read_csv(airlines_csv)
countries_df = pd.read_csv(countries_csv)
planes_df = pd.read_csv(planes_csv)
routes_df = pd.read_csv(routes_csv)
terminals_df = pd.read_csv(terminals_csv)

In [430]:
# for each df replace '\N' with None
airlines_df = airlines_df.map(lambda x: None if x == "\\N" else x)
countries_df = countries_df.map(lambda x: None if x == "\\N" else x)
planes_df = planes_df.map(lambda x: None if x == "\\N" else x)
routes_df = routes_df.map(lambda x: None if x == "\\N" else x)
terminals_df = terminals_df.map(lambda x: None if x == "\\N" else x)

In [431]:
# drop None values
airlines_df = airlines_df.dropna()
countries_df = countries_df.dropna()
planes_df = planes_df.dropna()
routes_df = routes_df.dropna()
terminals_df = terminals_df.dropna()

## Countries

In [432]:
countries_df

,name,iso_code,dafif_code
1,Aruba,AW,AA
2,Antigua and Barbuda,AG,AC
3,United Arab Emirates,AE,AE
4,Afghanistan,AF,AF
5,Algeria,DZ,AG
...,...,...,...
256,Samoa,WS,WS
257,Eswatini,SZ,WZ
258,Yemen,YE,YM
259,Zambia,ZM,ZA


In [433]:
# upload each country to the db
for _, row in tqdm(countries_df.iterrows(), total=len(countries_df)):
    with driver.session() as session:
        session.run(
            "MERGE (c:Country {name: $name, iso_code: $iso_code, dafif_code: $dafif_code})",
            name=row["name"],
            iso_code=row["iso_code"],
            dafif_code=row["dafif_code"],
        )

  0%|          | 0/240 [00:00<?, ?it/s]

## Airlines

In [434]:
airlines_df

,airline_id,name,alias,iata,icao,callsign,country,active
324,324,All Nippon Airways,ANA All Nippon Airways,NH,ANA,ALL NIPPON,Japan,Y
576,576,AirAsia,Air Asia,AK,AXM,ASIAN EXPRESS,Malaysia,Y
641,641,Rossiya-Russian Airlines,Pulkovo Aviation Enterprise,FV,SDM,PULKOVO,Russia,Y
1436,1437,bmi,bmi British Midland,BD,BMA,MIDLAND,United Kingdom,Y
1530,1531,Brussels Airlines,SN Brussels Airlines,SN,DAT,BEE-LINE,Belgium,Y
...,...,...,...,...,...,...,...,...
6028,19774,Spike Airlines,Aero Spike,S0,SAL,Spike Air,United States,Y
6063,19974,SOCHI AIR EXPRESS,ADLER EXPRESS,Q4,SAE,ADLER EXPRESS,Russia,Y
6096,20285,WestJet Encore,Encore,WR,WEN,Encore,Canada,Y
6153,21131,All America BOPY,All America BOPY,0P,PYB,BOPY,Paraguay,Y


In [435]:
airlines_df.columns

Index(['airline_id', 'name', 'alias', 'iata', 'icao', 'callsign', 'country',
       'active'],
      dtype='object')

In [436]:
with driver.session() as session:
    for _, row in tqdm(airlines_df.iterrows(), total=len(airlines_df)):
            session.run(
                "MERGE (a:Airline {id: $airline_id, name: $name, alias: $alias, iata: $iata, icao: $icao, callsign: $callsign, active: $active, country: $country})",
                airline_id=row["airline_id"],
                name=row["name"],
                alias=row["alias"],
                iata=row["iata"],
                icao=row["icao"],
                callsign=row["callsign"],
                country=row["country"],
                active=row["active"],
            )
            # math the airline with its country
            session.run(
                """
                MATCH (a:Airline {name: $name}),
                      (c:Country {name: $country})
                MERGE (a)-[:BASED_FROM]->(c)
                """,
                name=row["name"],
                country=row["country"]
            )

  0%|          | 0/84 [00:00<?, ?it/s]

## Planes

In [437]:
planes_df

,name,iata,icao
0,Aerospatiale (Nord) 262,ND2,N262
1,Aerospatiale (Sud Aviation) Se.210 Caravelle,CRV,S210
2,Aerospatiale SN.601 Corvette,NDC,S601
3,Aerospatiale/Alenia ATR 42-300,AT4,AT43
4,Aerospatiale/Alenia ATR 42-500,AT5,AT45
...,...,...,...
240,Tupolev Tu-134,TU3,T134
242,Tupolev Tu-154,TU5,T154
243,Tupolev Tu-204,T20,T204
244,Yakovlev Yak-40,YK4,YK40


In [438]:
planes_df.columns

Index(['name', 'iata', 'icao'], dtype='object')

In [439]:
with driver.session() as session:
    for _, row in tqdm(planes_df.iterrows(), total=len(planes_df)):
            session.run(
                "MERGE (p:PlaneType {name: $name, iata: $iata, icao: $icao})",
                name=row["name"],
                iata=row["iata"],
                icao=row["icao"],
            )

  0%|          | 0/220 [00:00<?, ?it/s]

## Terminals

In [440]:
terminals_df

,airport_id,name,city,country,iata,icao,latitude,longitude,altitude,timezone,dst,tz,type,source
0,1,Goroka Airport,Goroka,Papua New Guinea,GKA,AYGA,-6.081690,145.391998,5282,10,U,Pacific/Port_Moresby,airport,OurAirports
1,2,Madang Airport,Madang,Papua New Guinea,MAG,AYMD,-5.207080,145.789001,20,10,U,Pacific/Port_Moresby,airport,OurAirports
2,3,Mount Hagen Kagamuga Airport,Mount Hagen,Papua New Guinea,HGU,AYMH,-5.826790,144.296005,5388,10,U,Pacific/Port_Moresby,airport,OurAirports
3,4,Nadzab Airport,Nadzab,Papua New Guinea,LAE,AYNZ,-6.569803,146.725977,239,10,U,Pacific/Port_Moresby,airport,OurAirports
4,5,Port Moresby Jacksons International Airport,Port Moresby,Papua New Guinea,POM,AYPY,-9.443380,147.220001,146,10,U,Pacific/Port_Moresby,airport,OurAirports
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8494,9935,Kualanamu International Airport,Medan,Indonesia,KNO,WIMM,3.642222,98.885278,23,7,N,Asia/Jakarta,airport,OurAirports
8672,10114,Chichen Itza International Airport,Chichen Itza,Mexico,CZA,MMCT,20.641300,-88.446198,102,-6,S,America/Mexico_City,airport,OurAirports
8693,10135,San Cristobal de las Casas Airport,San Cristobal de las Casas,Mexico,SZT,MMSC,16.690300,-92.530098,7707,-6,N,America/Mexico_City,airport,OurAirports
8713,10156,Yeysk Airport,Eysk,Russia,EIK,URKE,46.680000,38.210000,60,3,N,Europe/Moscow,airport,OurAirports


In [441]:
terminals_df.columns

Index(['airport_id', 'name', 'city', 'country', 'iata', 'icao', 'latitude',
       'longitude', 'altitude', 'timezone', 'dst', 'tz', 'type', 'source'],
      dtype='object')

In [442]:
with driver.session() as session:
    for _, row in tqdm(terminals_df.iterrows(), total=len(terminals_df)):
        height_meters = row["altitude"] * 0.3048  # convert feet to meters
        session.run(
            "MERGE (t:Terminal {airport_id: $airport_id, name: $name, iata: $iata, icao: $icao, type: $type, source: $source, location: point({latitude: $latitude, longitude: $longitude, height: $height_meters})})",
            airport_id=row["airport_id"],
            name=row["name"],
            # city=row["city"],
            # country=row["country"],
            iata=row["iata"],
            icao=row["icao"],
            latitude=row["latitude"],
            longitude=row["longitude"],
            altitude=row["altitude"],
            timezone=row["timezone"],
            # dst=row["dst"],
            # tz=row["tz"],
            type=row["type"],
            source=row["source"],
            height_meters=height_meters,
        )
        # create timezone node
        session.run("MERGE (tz:Timezone {name: $name})", name=row["timezone"])

        # create city node
        session.run("MERGE (c:City {name: $city})", city=row["city"])
        # match city with timezone
        session.run(
            """
            MATCH (c:City {name: $city}),
                  (tz:Timezone {name: $timezone})
            MERGE (c)-[:IN_TIMEZONE]->(tz)
            """,
            city=row["city"],
            timezone=row["timezone"],
        )
        # match terminal with timezone
        session.run(
            """
            MATCH (t:Terminal {iata: $iata}),
                  (tz:Timezone {name: $timezone})
            MERGE (t)-[:IN_TIMEZONE]->(tz)
            """,
            iata=row["iata"],
            timezone=row["timezone"],
        )
        # match terminal with city
        session.run(
            """
            MATCH (t:Terminal {iata: $iata}),
                  (c:City {name: $city})
            MERGE (t)-[:LOCATED_IN]->(c)
            """,
            iata=row["iata"],
            city=row["city"],
        )
        # match terminal with country
        session.run(
            """
            MATCH (t:Terminal {iata: $iata}),
                  (c:Country {name: $country})
            MERGE (t)-[:LOCATED_IN]->(c)
            """,
            iata=row["iata"],
            country=row["country"],
        )
        # match city with country
        session.run(
            """
            MATCH (c:City {name: $city}),
                  (c:Country {name: $country})
            MERGE (c)-[:LOCATED_IN]->(c)
            """,
            city=row["city"],
            country=row["country"],
        )

  0%|          | 0/5586 [00:00<?, ?it/s]

## Routes

In [443]:
routes_df

,airline,airline_id,source_airport,source_airport_id,destination_airport,destination_airport_id,codeshare,stops,equipment
187,2P,897,GES,2402,MNL,2397,Y,0,320
197,2P,897,MNL,2397,GES,2402,Y,0,320
1059,4M,3201,DFW,3670,EZE,3988,Y,0,777
1060,4M,3201,EZE,3988,DFW,3670,Y,0,777
1061,4M,3201,EZE,3988,JFK,3797,Y,0,777
...,...,...,...,...,...,...,...,...,...
67429,ZH,4611,ZHA,6357,PEK,3364,Y,0,737
67431,ZH,4611,ZUH,6355,CKG,3393,Y,0,737
67432,ZH,4611,ZUH,6355,CTU,3395,Y,0,319
67436,ZH,4611,ZUH,6355,PEK,3364,Y,0,737 320


In [444]:
routes_df.columns

Index(['airline', 'airline_id', 'source_airport', 'source_airport_id',
       'destination_airport', 'destination_airport_id', 'codeshare', 'stops',
       'equipment'],
      dtype='object')

In [445]:
with driver.session() as session:
    for _, row in tqdm(routes_df.iterrows(), total=len(routes_df)):
        if row["stops"] > 1:
            continue
        session.run(
            """
            MATCH (source:Terminal {iata: $source_airport}),
                  (destination:Terminal {iata: $destination_airport}),
                  (airline:Airline {iata: $airline})
            MERGE (source)-[:FLIGHT_TO {iata: $airline, distance: point.distance(source.location, destination.location), plane: $equipment}]->(destination)
            """,
            source_airport=row["source_airport"],
            destination_airport=row["destination_airport"],
            airline=row["airline"],
            # stops=row["stops"],
            equipment=row["equipment"],
        )
        # match equipment to airline
        session.run(
            """
            MATCH (p:PlaneType {iata: $equipment}),
                  (a:Airline {iata: $airline})
            MERGE (a)-[:OPERATES]->(p)
            """,
            equipment=row["equipment"],
            airline=row["airline"],
        )

  0%|          | 0/14557 [00:00<?, ?it/s]

# Functions

In [446]:
# with driver.session() as session:
#     session.run(
#         """
#
#         """)